<a href="https://colab.research.google.com/github/MateoUsme/Esp_Analitica/blob/main/04%20-%20%5BTALLER%5D%20-%20MAP%20REDUCE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/UDEA-Esp-Analitica-y-Ciencia-de-Datos/EACD-03-BIGDATA/master/init.py
import init; init.init(force_download=False); 
from IPython.display import Image

replicating local resources


In [ ]:
Image("local/imgs/udea-datascience.png")

# **TALLER 1: MAP REDUCE**
## Desarrollado por: Mg. Efraín Alberto Oviedo
## Universidad de Antioquia

Por favor, ingrese la información de cada integrante del grupo de trabajo

**Nombre Completo:**

**Cédula:**

**Correo Institucional:**

**Nombre Completo:**

**Cédula:**

**Correo Institucional:**

**Nombre Completo:**

**Cédula:**

**Correo Institucional:**

**Nombre Completo:**

**Cédula:**

**Correo Institucional:**

#**Análisis de Accidentalidad en la ciudad de Medellín**

La secretaría de movilidad de la alcaldía de Medellín ha recolectado datos relacionados con la accidentalidad vial del año 2017 (Disponible en https://www.datos.gov.co) registrando en cada accidente la siguiente información: 
* Fecha
* Hora
* Día de la semana
* Clase de accidente
* Dirección
* Gravedad
* Barrio
* Comuna

Con el fin de adelantar programas que permitan disminuir la accidentalidad vial en la ciudad, se requiere de su ayuda para generar un informe que contenga la siguiente información:



In [5]:
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop3.2.tgz
!tar xf spark-3.0.1-bin-hadoop3.2.tgz 
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.1-bin-hadoop3.2"

In [7]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkContext
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = SparkContext.getOrCreate()

##**A) Porcentaje de accidentes en cada día de la semana**


In [9]:
data = sc.textFile("local/data/accidentalidad.csv").map(lambda x:x.split(","))
data_hdr = data.first()
data = data.filter(lambda x: x!=data_hdr)
data.take(5)

[['1565221',
  '01/01/2017',
  '00:10:00',
  'DOMINGO  ',
  'Atropello',
  'CL 68 CR 87',
  'EPM con Interior',
  'HERIDO',
  'Palenque',
  'Robledo',
  'Tramo de via'],
 ['1565189',
  '01/01/2017',
  '00:20:00',
  'DOMINGO  ',
  'Choque',
  'CL 44 CR 93',
  'EPM sin Interior',
  'HERIDO',
  'Campo Alegre',
  'La América',
  'Tramo de via'],
 ['1565182',
  '01/01/2017',
  '00:20:00',
  'DOMINGO  ',
  'Choque',
  'CR 16 CL 56',
  'Malla vial',
  'HERIDO',
  'Villatina',
  'Villa Hermosa',
  'Tramo de via'],
 ['1565404',
  '01/01/2017',
  '01:25:00',
  'DOMINGO  ',
  'Choque',
  'CR 41 CL 85',
  'Malla vial',
  'SOLO DAÑOS',
  'Las Granjas',
  'Manrique',
  'Interseccion'],
 ['1565185',
  '01/01/2017',
  '01:30:00',
  'DOMINGO  ',
  'Choque',
  'CR 76 CL 91',
  'Malla vial',
  'HERIDO',
  'Kennedy',
  'Doce de Octubre',
  'Tramo de via']]

In [37]:
ans2 = data.map(lambda x: (x[3],1)).reduceByKey(lambda a,b: a+b)
totaaal = (ans2.values().sum())**-1
totalito = ans2.mapValues(lambda x: f"{x*totaaal*100:.0f}%")
totalito.collect()

[('DOMINGO  ', '9%'),
 ('MARTES   ', '16%'),
 ('MIÉRCOLES', '16%'),
 ('LUNES    ', '14%'),
 ('JUEVES   ', '15%'),
 ('VIERNES  ', '16%'),
 ('SÁBADO   ', '14%')]

##**B) Barrio en el que se presenta el mayor y el menor número de accidentes**

In [40]:
ans1 = data.map(lambda x: (x[-3],1)).reduceByKey(lambda a,b: a+b)
print(f"Barrio con mayor nro: {ans1.max(lambda x:x[1])}\n\nBarrio con menor nro: {ans1.min(lambda x:x[1])}")

Barrio con mayor nro: ('La Candelaria', 944)

Barrio con menor nro: ('Villa Lilliam', 1)


##**C) Comuna en la que se presenta mayor número de accidentes con muertos**

In [44]:
ans3 = data.filter(lambda x: x[7].upper() == "MUERTO").map(lambda x: (x[-2],1)).reduceByKey(lambda a,b: a+b)
ans3.takeOrdered(3,key=lambda x:-x[1])

[('La Candelaria', 14), ('Guayabal', 11), ('Castilla', 10)]

##**D)	Día de la semana en que se presentaron mayor cantidad de accidentes con muertos**

In [47]:
ans4 = data.filter(lambda x: x[7].upper() == "MUERTO").map(lambda x: (x[3].replace(" ",""),1)).reduceByKey(lambda a,b: a+b)
ans4.takeOrdered(3,key=lambda x:-x[1])

[('DOMINGO', 31), ('SÁBADO', 28), ('LUNES', 24)]

##**E)	Hora exacta y rango de hora en la que se presentó mayor cantidad de accidentes con heridos**

Presente aquí el código desarrollado y el resultado obtenido

#**Propuesta para disminuir la accidentalidad de la ciudad**

A la luz de los resultados obtenidos anteriormente, presente una propuesta orientada a mejorar las cifras de la accidentalidad en la ciudad.